In [ ]:
C:\Users\coldbrew\VTON-project\data\dataset\image

In [2]:

import os, sys

notebook_dir = os.getcwd()
project_root_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
dataset_person_folder = os.path.join(project_root_dir, "data", "dataset", "image")
dataset_person_upper_mask_folder = os.path.join(project_root_dir, "data", "dataset", "image_mask_U")
dataset_person_lower_mask_folder = os.path.join(project_root_dir, "data", "dataset", "image_mask_L")

dst_person_with_upper_agnostic_folder = os.path.join(project_root_dir, "data", "dataset", "image_with_upper_agnostic")
dst_person_with_lower_agnostic_folder = os.path.join(project_root_dir, "data", "dataset", "image_with_lower_agnostic")

# 결과 저장 폴더가 없으면 생성
os.makedirs(dst_person_with_upper_agnostic_folder, exist_ok=True)
os.makedirs(dst_person_with_lower_agnostic_folder, exist_ok=True)

# 사람이미지에 각각 마스크 이미지 넣어서 저장
import cv2
import numpy as np
from tqdm import tqdm

# 사람 이미지 파일 목록 가져오기 (마스크 이미지와 파일명이 동일하다고 가정)
person_files = sorted(os.listdir(dataset_person_folder))

for file_name in tqdm(person_files, desc="Processing images"):
    # 파일 경로 구성
    person_img_path = os.path.join(dataset_person_folder, file_name)
    upper_mask_path = os.path.join(dataset_person_upper_mask_folder, file_name)
    lower_mask_path = os.path.join(dataset_person_lower_mask_folder, file_name)
    
    # 이미지 로드 (마스크는 그레이스케일로 로드)
    person_img = cv2.imread(person_img_path)
    upper_mask = cv2.imread(upper_mask_path, cv2.IMREAD_GRAYSCALE)
    lower_mask = cv2.imread(lower_mask_path, cv2.IMREAD_GRAYSCALE)
    
    if person_img is None:
        print(f"Error loading person image: {file_name}")
        continue
    if upper_mask is None:
        print(f"Error loading upper mask image: {file_name}")
        continue
    if lower_mask is None:
        print(f"Error loading lower mask image: {file_name}")
        continue

    # 상체 마스크 적용: 마스크 영역을 검정색(0)으로 채우기
    upper_agnostic_img = person_img.copy()
    # binary 마스크라고 가정 (0: 배경, 255: 마스크 영역). 임계값 127을 기준으로 처리합니다.
    upper_agnostic_img[upper_mask > 127] = 0

    # 하체 마스크 적용: 마스크 영역을 검정색(0)으로 채우기
    lower_agnostic_img = person_img.copy()
    lower_agnostic_img[lower_mask > 127] = 0

    # 결과 저장
    cv2.imwrite(os.path.join(dst_person_with_upper_agnostic_folder, file_name), upper_agnostic_img)
    cv2.imwrite(os.path.join(dst_person_with_lower_agnostic_folder, file_name), lower_agnostic_img)

Processing images: 100%|██████████| 195/195 [00:02<00:00, 93.67it/s]


In [3]:
import os, sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
catvton_dir = os.path.join(parent_dir, "CatVTON")
if catvton_dir not in sys.path:
    sys.path.insert(0, catvton_dir)

In [ ]:
import os
from huggingface_hub import snapshot_download
from tqdm import tqdm
from model.cloth_masker import AutoMasker
import matplotlib.pyplot as plt
import PIL.Image

# 데이터셋 경로 설정 (노트북에서 직접 입력)
data_root_path = "/Users/coldbrew/Documents/VTON-project/dataset"  # Google Colab에서 사용 예시, 로컬 경로로 변경 가능
repo_path = "zhengchong/CatVTON"  # CatVTON 모델 레포

# GPU 사용 확인 (CUDA 사용 가능 여부)
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# CatVTON 모델 다운로드
repo_path = snapshot_download(repo_id=repo_path)
print(f"Model downloaded to: {repo_path}")

# AutoMasker 초기화 (DensePose와 SCHP 사용)
automasker = AutoMasker(
    densepose_ckpt=os.path.join(repo_path, "DensePose"),
    schp_ckpt=os.path.join(repo_path, "SCHP"),
    device=device,
)

# 이미지 폴더 경로
images_dir = os.path.join(data_root_path, "images")
assert os.path.exists(images_dir), f"Images folder {images_dir} does not exist."

# 상의 및 하의 마스크를 저장할 출력 디렉토리 생성
upper_output_dir = os.path.join(data_root_path, "upper_agnostic_masks")
lower_output_dir = os.path.join(data_root_path, "lower_agnostic_masks")
os.makedirs(upper_output_dir, exist_ok=True)
os.makedirs(lower_output_dir, exist_ok=True)

# images 폴더의 모든 .jpg 파일 처리
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
print(f"Found {len(image_files)} .jpg files in {images_dir}")

for image_file in tqdm(image_files, desc="Processing images"):
    image_path = os.path.join(images_dir, image_file)
    
    # 상의 마스크 생성 및 저장
    upper_mask = automasker(
        image_path,
        maks_type='upper'
    )['mask']
    upper_mask_filename = image_file.replace('.jpg', '_upper_mask.png')
    upper_mask.save(os.path.join(upper_output_dir, upper_mask_filename))
    print(f"Saved upper agnostic mask to {os.path.join(upper_output_dir, upper_mask_filename)}")

    # 하의 마스크 생성 및 저장
    lower_mask = automasker(
        image_path,
        maks_type='lower'
    )['mask']
    lower_mask_filename = image_file.replace('.jpg', '_lower_mask.png')
    lower_mask.save(os.path.join(lower_output_dir, lower_mask_filename))
    print(f"Saved lower agnostic mask to {os.path.join(lower_output_dir, lower_mask_filename)}")

    # 생성된 마스크 시각화 (선택 사항)
    upper_mask_img = PIL.Image.open(os.path.join(upper_output_dir, upper_mask_filename))
    lower_mask_img = PIL.Image.open(os.path.join(lower_output_dir, lower_mask_filename))

    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(upper_mask_img, cmap='gray')
    plt.title(f"Upper Mask - {image_file}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(lower_mask_img, cmap='gray')
    plt.title(f"Lower Mask - {image_file}")
    plt.axis('off')

    plt.show()

/Users/coldbrew/anaconda3/envs/torch_book/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/coldbrew/anaconda3/envs/torch_book/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /Users/coldbrew/anaconda3/envs/torch_book/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/Users/coldbrew/anaconda3/envs/torch_book/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/coldbrew/anaconda3/envs/torch_book/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/coldbrew/anaconda3/envs/torch_book/lib/libjpeg.9.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/coldbrew/anaconda3/envs/torch_book/lib/libjpeg.9.dylib' (no such file), '/Users/coldbrew/anaconda3/envs/torch_book/lib/libjpeg.9.dylib' (no such file), '/Syst

Using device: cpu


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Model downloaded to: /Users/coldbrew/.cache/huggingface/hub/models--zhengchong--CatVTON/snapshots/2969fcf85fe62f2036605716f0b56f0b81d01d79


AssertionError: Torch not compiled with CUDA enabled